In [1]:
#import libraries
from time import clock
from osgeo import gdal
import os , sys


In [2]:
r_data_dir = "SampleHDFData"
data_dir = "SampleHDFData"
files_names = [f for f in os.listdir(r_data_dir) if f.endswith('.hdf')]
files_names

['MCD64A1.A2009032.h10v05.006.2017015033638.hdf',
 'MCD64A1.A2009001.h10v05.006.2017015111541.hdf',
 'MCD64A1.A2009060.h10v05.006.2017015044612.hdf']

In [18]:
# loop over files
for i in files_names:
    print("Processing file: " + i)
    print(" ")
    # create file path for the hdf file
    name =  "\"HDF4_EOS:EOS_GRID:" + data_dir + "/" + i + ":MOD_Grid_Monthly_500m_DB_BA:Burn Date\""
    
    # extract a substring from the filename
    b = name[45:53]+ "_"+ name[54:59]
    
    # get the time the image was taken
    year = name[46:51]
    
    # create output directories for the rasters if necessary
    if not os.path.exists(r_data_dir + "/" + year):
        dir_to_make = r_data_dir + "/" + year
        print("Making the following directory: " + dir_to_make)
        os.makedirs(dir_to_make)
    
    # convert the files to GeoTIFF format
    dset = data_dir + "/" + year + "/" + b +".tif"
    cmd = "gdal_translate -of GTiff " + name + " " + dset
    print("HDF to GeoTIFF command: " + cmd)
    os.system(cmd)

    r = gdal.Open(dset)
    print("Old projection: " + r.GetProjection())
    print(" ")
    
    # reproject the raster to a new projection, save as new file
    new_file = data_dir + "/" + year + "/" + b +"_projected.tif"
    reproj_cmd = "gdalwarp -t_srs '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=37.5 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs ' " + dset + " " + new_file
    print("Reprojection command: " + reproj_cmd)
    os.system(reproj_cmd)
    
    print(" ")
    
    # load the raster 
    r = gdal.Open(new_file)
    print("New projection: " + r.GetProjection())
    print(" ")
    
    print("---------")

Processing file: MCD64A1.A2009032.h10v05.006.2017015033638.hdf
 
HDF to GeoTIFF command: gdal_translate -of GTiff "HDF4_EOS:EOS_GRID:SampleHDFData/MCD64A1.A2009032.h10v05.006.2017015033638.hdf:MOD_Grid_Monthly_500m_DB_BA:Burn Date" SampleHDFData/032.h/9032.h10_05.00.tif
Old projection: PROJCS["unnamed",GEOGCS["Unknown datum based upon the custom spheroid",DATUM["Not_specified_based_on_custom_spheroid",SPHEROID["Custom spheroid",6371007.181,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]
 
Reprojection command: gdalwarp -t_srs '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=37.5 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs ' SampleHDFData/032.h/9032.h10_05.00.tif SampleHDFData/032.h/9032.h10_05.00_projected.tif
 
New projection: PROJCS["unnamed",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHE